# Implementation of SELUs & Dropout-SELUs in NumPy
This looks pretty neat. 
They can prove that when you slightly modify the ELU activation,
your average unit activation goes towards zero mean/unit variance (if the network is deep enough). 
If they're right, this might make batch norm obsolete, which would be a huge bon to training speeds! 

The experiments look convincing, so apparently it even beats BN+ReLU in accuracy... though 

I wish they would've shown the resulting distributions of activations after training. 

But assuming their fixed point proof is true, it will. 

Still, still would've been nice if they'd shown it -- maybe they ran out of space in their appendix ;)

Weirdly, the exact ELU modification they proposed isn't stated explicitly in the paper! 

For those wondering, it can be found in the available sourcecode, and looks like this:

In [1]:
# An extra explaination from Reddit
# # Thanks, I will double check the analytical solution. For the numerical one, could you please explain why running the following code results in a value close to 1 rather than 0?
# du = 0.001
# u_old = np.mean(selu(np.random.normal(0,    1, 100000000)))
# u_new = np.mean(selu(np.random.normal(0+du, 1, 100000000)))
# # print (u_new-u_old) / du
# print(u_old, u_new)
# # Now I see your problem: 
# #     You do not consider the effect of the weights. 
# #     From one layer to the next, we have two influences: 
# #         (1) multiplication with weights and 
# #         (2) applying the SELU. 
# #         (1) has a centering and symmetrising effect (draws mean towards zero) and 
# #         (2) has a variance stabilizing effect (draws variance towards 1). 

# #         That is why we use the variables \mu&\omega and \nu&\tau to analyze the both effects.
# # Oh yes, thats true, zero mean weights completely kill the mean. Thanks!

# Tensorflow implementation
import numpy as np

def selu(x):
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    return scale * np.where(x>=0.0, x, alpha * (np.exp(x)-1))

In [2]:
# # Tensorflow implementation on github
# def dropout_selu(x, rate, alpha= -1.7580993408473766, fixedPointMean=0.0, fixedPointVar=1.0, 
#                  noise_shape=None, seed=None, name=None, training=False):
#     """Dropout to a value with rescaling."""

#     def dropout_selu_impl(x, rate, alpha, noise_shape, seed, name):
#         keep_prob = 1.0 - rate
#         x = ops.convert_to_tensor(x, name="x")
#         if isinstance(keep_prob, numbers.Real) and not 0 < keep_prob <= 1:
#             raise ValueError("keep_prob must be a scalar tensor or a float in the "
#                                              "range (0, 1], got %g" % keep_prob)
#         keep_prob = ops.convert_to_tensor(keep_prob, dtype=x.dtype, name="keep_prob")
#         keep_prob.get_shape().assert_is_compatible_with(tensor_shape.scalar())

#         alpha = ops.convert_to_tensor(alpha, dtype=x.dtype, name="alpha")
#         keep_prob.get_shape().assert_is_compatible_with(tensor_shape.scalar())

#         if tensor_util.constant_value(keep_prob) == 1:
#             return x

#         noise_shape = noise_shape if noise_shape is not None else array_ops.shape(x)
#         random_tensor = keep_prob
#         random_tensor += random_ops.random_uniform(noise_shape, seed=seed, dtype=x.dtype)
#         binary_tensor = math_ops.floor(random_tensor)
#         ret = x * binary_tensor + alpha * (1-binary_tensor)

#         a = tf.sqrt(fixedPointVar / (keep_prob *((1-keep_prob) * tf.pow(alpha-fixedPointMean,2) + fixedPointVar)))

#         b = fixedPointMean - a * (keep_prob * fixedPointMean + (1 - keep_prob) * alpha)
#         ret = a * ret + b
#         ret.set_shape(x.get_shape())
#         return ret

#     with ops.name_scope(name, "dropout", [x]) as name:
#         return utils.smart_cond(training,
#             lambda: dropout_selu_impl(x, rate, alpha, noise_shape, seed, name),
#             lambda: array_ops.identity(x))

In [3]:
# """"""""""""""""""""""""""""""""""""""Dropout to a value with rescaling."""""""""""""""""""""""""""""""""""""""
# NumPy implementation
def dropout_selu(X, p_dropout):
    alpha= -1.7580993408473766 
    fixedPointMean=0.0 
    fixedPointVar=1.0
    keep_prob = 1.0 - p_dropout
#     noise_shape = noise_shape.reshape(X)
    random_tensor = keep_prob
    #         random_tensor += random_ops.random_uniform(noise_shape, seed=seed, dtype=x.dtype)
    random_tensor += np.random.uniform(size=X.shape) # low=0, high=1
    #         binary_tensor = math_ops.floor(random_tensor)
    binary_tensor = np.floor(random_tensor)
    ret = X * binary_tensor + alpha * (1-binary_tensor)

    #         a = tf.sqrt(fixedPointVar / (keep_prob *((1-keep_prob) * tf.pow(alpha-fixedPointMean,2) + fixedPointVar)))
    a = np.sqrt(fixedPointVar / (keep_prob *((1-keep_prob) * ((alpha-fixedPointMean)**2) + fixedPointVar)))

    b = fixedPointMean - a * (keep_prob * fixedPointMean + (1 - keep_prob) * alpha)
    ret = a * ret + b
    #         ret.set_shape(x.get_shape())
    ret = ret.reshape(X.shape)
    return ret

In [26]:
# EDIT: For the fun of it, I ran a quick experiment to see if activations would really stay close to 0/1:
x = np.random.normal(size=(300, 200))
for _ in range(100):
    w = np.random.normal(size=(200, 200), scale=np.sqrt(1/200))  # their initialization scheme
    x = x @ w
    x = selu(x=x)
    x = dropout_selu(X=x, p_dropout=0.10)
    mean = x.mean(axis=1)
    scale = x.std(axis=1) # standard deviation=square-root(variance)
    print(mean.min(), mean.max(), scale.min(), scale.max())

-0.269529622603 0.238741461412 0.842533466259 1.12314077973
-0.191276680836 0.196408512881 0.863940928665 1.15999659563
-0.260305796052 0.198248391232 0.840376306318 1.16079631406
-0.178038627949 0.195871492111 0.829389516362 1.15729035683
-0.184791099668 0.247544538154 0.86009545706 1.17806478766
-0.184567678178 0.176109012134 0.883795665608 1.15019045197
-0.177685947525 0.272610135699 0.857025015242 1.1560739998
-0.172413310645 0.251602544631 0.866192464996 1.14706840521
-0.204783155251 0.196256467718 0.831413789433 1.12569688276
-0.231260917235 0.199345543742 0.850621042677 1.15488294415
-0.198262291455 0.185763410598 0.86570902624 1.18618903187
-0.189368179065 0.207968257914 0.849682926164 1.19718414045
-0.245751492647 0.182934599357 0.8674369991 1.13325946801
-0.193985688812 0.180393480031 0.842656664173 1.18722701691
-0.189262554415 0.28281139756 0.866088293902 1.13905094872
-0.214843349707 0.197344476764 0.863050346612 1.1639797022
-0.203057058846 0.194591558071 0.853361773197 1

In [7]:
# My NumPy implemetation of Normal dropout for ReLU
def dropout_forward(X, p_dropout):
    u = np.random.binomial(1, p_dropout, size=X.shape) / p_dropout
    out = X * u
    cache = u
    return out, cache

def dropout_backward(dout, cache):
    dX = dout * cache
    return dX

In [8]:
# EDIT: For the fun of it, I ran a quick experiment to see if activations would really stay close to 0/1:
x = np.random.normal(size=(300, 200))
for _ in range(100):
    w = np.random.normal(size=(200, 200), scale=np.sqrt(1/200))  # their initialization scheme
    x = x @ w
    x = selu(x)
    x, _ = dropout_forward(p_dropout=0.8, X=x)
    mean = x.mean(axis=1)
    scale = x.std(axis=1) # standard deviation=square-root(variance)
    print(mean.min(), mean.max(), scale.min(), scale.max())

-0.214012249027 0.229167129326 0.932332542517 1.35514712732
-0.223693792421 0.356189784779 1.00770587731 1.49948911469
-0.205881861505 0.328283949271 1.03819915898 1.54480701192
-0.237758376677 0.305559993102 1.1349800102 1.67391420544
-0.236016430438 0.391877470169 1.09925499753 1.8112463819
-0.18195362093 0.496330978983 1.21282794598 1.89902328039
-0.171558426569 0.412001003287 1.19432537679 2.06609202396
-0.16823370468 0.467072835519 1.19269352637 2.15085026297
-0.212979175376 0.547603855662 1.21138654432 2.15184578053
-0.177900749039 0.6419724667 1.20386863829 2.13084117537
-0.202725010062 0.495420764904 1.18244872568 2.24586723365
-0.123165700297 0.5420187544 1.29085802962 2.19068147843
-0.251513671944 0.551490812535 1.31297112664 2.07115535574
-0.182744872091 0.458915855254 1.24893027974 2.14046098202
-0.135270285146 0.580220120799 1.20291256989 2.19956243188
-0.14777285446 0.538440624983 1.26158617999 2.10253823161
-0.18342114927 0.421910974881 1.26314496206 2.03972945366
-0.149

In [6]:
def elu_fwd(X):
    alpha = 1.0
    scale = 1.0
    #     return scale * np.where(x>=0.0, x, alpha * (np.exp(x)-1))
    X_pos = np.maximum(0.0, X) # ReLU
    X_neg = np.minimum(X, 0.0) # otherwise: if X<=0, Exp Leaky ReLU
    X_neg_exp = alpha * (np.exp(X_neg)-1) # a: slope, a>=0
    out = scale * (X_pos + X_neg_exp)
    cache = (scale, alpha, X) # mean=0, std=1
    return out, cache

def elu_bwd(dout, cache):
    scale, alpha, X = cache # mean=0, std=1
    dout = dout * scale
    dX_neg = dout.copy()
    dX_neg[X>0] = 0
    X_neg = np.minimum(X, 0) # otherwise: if X<=0, Exp Leaky ReLU
    dX_neg = dX_neg * alpha * np.exp(X_neg) # derivative of abs(np.exp(X_neg)-1) # a: slope, a>=0
    dX_pos = dout.copy()
    dX_pos[X<0] = 0
    dX_pos = dX_pos * 1
    dX = dX_neg + dX_pos
    return dX

In [7]:
# EDIT: For the fun of it, I ran a quick experiment to see if activations would really stay close to 0/1:
x = np.random.normal(size=(300, 200))
for _ in range(100):
    w = np.random.normal(size=(200, 200), scale=np.sqrt(1/200))  # their initialization scheme
    x = x @ w
    x, _ = elu_fwd(X=x)
    x, _ = dropout_forward(p_dropout=0.95, X=x)
    mean = x.mean(axis=1)
    scale = x.std(axis=1) # standard deviation=square-root(variance)
    print(mean.min(), mean.max(), scale.min(), scale.max())

0.0267840597416 0.34870387555 0.688048836569 1.04717887996
0.0118506955263 0.304876451832 0.56287559608 0.883068359607
-0.00848144962242 0.206902931306 0.461438275839 0.773605957189
-0.0135542733853 0.188593823674 0.382063563973 0.688186843237
-0.0561000469178 0.122238741339 0.349825172164 0.602874984924
-0.0343566384099 0.18028897461 0.31253884688 0.548510582005
-0.0429958073188 0.116054084819 0.258804537482 0.468348512243
-0.0365803448673 0.0872113156711 0.24577948731 0.438054082984
-0.0599072152755 0.107624519345 0.215066658884 0.422357677398
-0.0350411695585 0.0776162947341 0.205136251217 0.425153568286
-0.0283457218838 0.0652535851403 0.185845104457 0.385896334075
-0.0272042989458 0.0766230668132 0.17945437564 0.343158539221
-0.0265363719492 0.0529204998045 0.175905477592 0.337747682314
-0.0195131277272 0.050982043472 0.157488026364 0.320393452692
-0.0279429986826 0.0494597719809 0.148463686617 0.302291903009
-0.0163050932564 0.0395091568666 0.134573867209 0.275776177388
-0.035063

In [13]:
def selu_fwd(X):
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    #     return scale * np.where(x>=0.0, x, alpha * (np.exp(x)-1))
    X_pos = np.maximum(0.0, X) # ReLU
    X_neg = np.minimum(X, 0.0) # otherwise: if X<=0, Exp Leaky ReLU
    X_neg_exp = alpha * (np.exp(X_neg)-1) # a: slope, a>=0
    out = scale * (X_pos + X_neg_exp)
    cache = (scale, alpha, X) # mean=0, std=1
    return out, cache

def selu_bwd(dout, cache):
    scale, alpha, X = cache # mean=0, std=1
    dout = dout * scale
    dX_neg = dout.copy()
    dX_neg[X>0] = 0
    X_neg = np.minimum(X, 0) # otherwise: if X<=0, Exp Leaky ReLU
    dX_neg = dX_neg * alpha * np.exp(X_neg) # derivative of abs(np.exp(X_neg)-1) # a: slope, a>=0
    dX_pos = dout.copy()
    dX_pos[X<0] = 0
    dX_pos = dX_pos * 1
    dX = dX_neg + dX_pos
    return dX

# def dropout_selu_forward(X, p_dropout):
def dropout_selu_forward(X, keep_prob):
    alpha= -1.7580993408473766
    fixedPointMean=0.0
    fixedPointVar=1.0

    u = np.random.binomial(1, keep_prob, size=X.shape) / keep_prob
    out = X * u + alpha * (1-u)

    #     a = tf.sqrt(fixedPointVar / (keep_prob *((1-keep_prob) * tf.pow(alpha-fixedPointMean,2) + fixedPointVar)))
    a = np.sqrt(fixedPointVar / (keep_prob *((1-keep_prob) * (alpha-fixedPointMean)**2 + fixedPointVar)))
    b = fixedPointMean - a * (keep_prob * fixedPointMean + (1 - keep_prob) * alpha)
    out = a * out + b
    cache = a, u
    return out, cache

def dropout_selu_backward(dout, cache):
    a, u = cache
    dout = dout * a
    dX = dout * u
    return dX

In [14]:
# EDIT: For the fun of it, I ran a quick experiment to see if activations would really stay close to 0/1:
x = np.random.normal(size=(300, 200))
for _ in range(100):
    w = np.random.normal(size=(200, 200), scale=np.sqrt(1/200))  # their initialization scheme
    x = x @ w
    x, cache = selu_fwd(x)
    x, _ = dropout_selu_forward(keep_prob=0.95, X=x)
    mean = x.mean(axis=1)
    scale = x.std(axis=1) # standard deviation=square-root(variance)
    print(mean.min(), mean.max(), scale.min(), scale.max())

-0.115228899805 0.273307640366 0.832187931633 1.20092983661
-0.11342776269 0.297433766906 0.890738815289 1.25180513542
-0.12925098648 0.362425379672 0.93295919605 1.3384718603
-0.12964971994 0.366295910839 0.966780089222 1.40596996607
-0.0864575866013 0.290687936815 0.959321646025 1.33767307621
-0.111998627179 0.300401219852 0.951157571282 1.35759226264
-0.0878051364106 0.313732327559 0.985100281202 1.39437359224
-0.102829064867 0.313342590658 0.967987306932 1.41533396008
-0.134024418009 0.514400568221 1.0022174775 1.42117544315
-0.156572603248 0.331289658482 1.00391056547 1.41748889863
-0.164599296478 0.300678860193 0.987115048323 1.39145024166
-0.0764760393873 0.297173187184 0.996668365766 1.37470533887
-0.126296878605 0.326807280604 0.978965605922 1.37800377186
-0.0872685363087 0.326317075429 0.987838245488 1.40211446152
-0.118293785738 0.419532184593 0.958342284191 1.3946407666
-0.0860908759643 0.436037876788 0.961645016304 1.44609169824
-0.0636222554284 0.386101182923 0.9988063343

In [18]:
# EDIT: For the fun of it, I ran a quick experiment to see if activations would really stay close to 0/1:
x = np.random.normal(size=(300, 200))
for _ in range(100):
    w = np.random.normal(size=(200, 200), scale=np.sqrt(1/200))  # their initialization scheme
    x = x @ w
    x, cache = selu_fwd(x)
    x = dropout_selu(X=x, p_dropout=0.10)
    mean = x.mean(axis=1)
    scale = x.std(axis=1) # standard deviation=square-root(variance)
    print(mean.min(), mean.max(), scale.min(), scale.max())

-0.235271254088 0.218746820232 0.819007812136 1.11747800096
-0.20951946942 0.180946004293 0.834080921638 1.13113481879
-0.191676850317 0.224930764106 0.868843915156 1.13158722935
-0.176701747105 0.157517304949 0.852585573073 1.12639938655
-0.162840032431 0.186669002989 0.858098346654 1.18314823773
-0.220704901111 0.190378608271 0.833024054349 1.14342284633
-0.19923882051 0.211312399024 0.87131817578 1.121972573
-0.210321754749 0.237660213544 0.871612931827 1.11061683783
-0.185461154997 0.225793126856 0.862512256897 1.1125366835
-0.178912312266 0.155551393681 0.852681937939 1.16124534451
-0.208812346507 0.179387508362 0.84954272907 1.16233918417
-0.227677609213 0.203579291629 0.853329180904 1.13222261062
-0.177117268663 0.182170079468 0.870238280515 1.21209197618
-0.187281710031 0.206016154254 0.866706826485 1.24963262405
-0.18972440741 0.171150605527 0.839788078575 1.16848345882
-0.187734315865 0.204341560836 0.850645039618 1.14832907681
-0.280902837708 0.170151586913 0.888266070211 1.

# Discussion & wrapup
According to this, even after a 100 layers, mean neuron activations stay fairly close to mean 0 / variance 1 
(even the most extreme means/variances are only off by 0.2).

Sepp Hochreiter is amazing: LSTM, meta-learning, SNNN. 

I think he has already done a much larger contribution to science than some self-proclaimed pioneers of DL 
who spend more time on social networks than actually doing any good research.